# Three Methods of Prompt Configuration

This notebook demonstrates different approaches to configuring agent prompts, from simple strings to dynamic, context-aware prompts.

## Key Concepts
- **Method 1**: String Prompt (simplest)
- **Method 2**: SystemMessage (structured)
- **Method 3**: Callable/Dynamic Prompt (advanced)

## When to Use Each Method
- **String**: Simple, static agents
- **SystemMessage**: Production chat agents
- **Callable**: Adaptive, personalized agents

## Prerequisites

Make sure you have the required packages installed:

```bash
pip install langchain langchain-community langchain-core langgraph pydantic
ollama pull qwen3
ollama serve
```

In [ ]:
# Import required modules
from langchain_ollama import ChatOllama
from langchain.agents import create_agent
from langchain_core.messages import SystemMessage
import tools

## Method 1: Simple String Prompt

The simplest approach - direct string instruction to the agent.

In [ ]:
print("=== Method 1: Simple String Prompt ===")

# Create model instance
model = ChatOllama(model="qwen3")

# Method 1: String prompt (simplest)
agent1 = create_agent(
    model,
    [tools.helper_tool],
    prompt="You are a helpful assistant. Be concise and accurate in your responses."
)

print("✓ Agent 1 created with string prompt")
print("  Characteristics:")
print("  - Direct string instruction")
print("  - Simple and readable")
print("  - Good for basic use cases")
print("  - Easy to modify")

## Method 2: SystemMessage Prompt

Using LangChain's SystemMessage class for more structured prompt handling.

In [ ]:
print("=== Method 2: SystemMessage Prompt ===")

# Method 2: SystemMessage prompt (structured)
agent2 = create_agent(
    model,
    [tools.helper_tool],
    prompt=SystemMessage(
        content="You are a research assistant. Always cite your sources and provide detailed explanations."
    )
)

print("✓ Agent 2 created with SystemMessage prompt")
print("  Characteristics:")
print("  - Structured message object")
print("  - Better integration with chat models")
print("  - More explicit about message type")
print("  - Professional for production use")

## Method 3: Dynamic Callable Prompt

A function that generates prompts based on state - the most flexible approach.

In [ ]:
print("=== Method 3: Dynamic Callable Prompt ===")

# Method 3: Callable/Dynamic prompt (most flexible)
def dynamic_prompt(state):
    """Generate prompt based on user type and context."""
    user_type = state.get("user_type", "standard")
    
    if user_type == "expert":
        system_msg = SystemMessage(
            content="Provide detailed technical responses with code examples and advanced concepts."
        )
    elif user_type == "beginner":
        system_msg = SystemMessage(
            content="Provide simple, clear explanations suitable for beginners. Use analogies and avoid jargon."
        )
    else:  # standard
        system_msg = SystemMessage(
            content="Provide balanced explanations that are informative but accessible."
        )
    
    return [system_msg] + state["messages"]

agent3 = create_agent(model, [tools.helper_tool], prompt=dynamic_prompt)

print("✓ Agent 3 created with dynamic callable prompt")
print("  Characteristics:")
print("  - Adapts to user context")
print("  - Personalized responses")
print("  - Most flexible approach")
print("  - Perfect for adaptive systems")

## Testing All Three Agents

Let's test all agents with the same question to see their different behaviors:

In [ ]:
# Test question for all agents
test_question = "Help me understand artificial intelligence"

print(f"Testing all agents with: '{test_question}'")
print("=" * 70)

# Test Agent 1 (String prompt)
print("\n=== Agent 1 Response (String Prompt) ===")
print("Expected: Concise and accurate response")
try:
    result1 = agent1.invoke({"messages": test_question})
    print(f"Response: {result1['messages'][-1].content}")
except Exception as e:
    print(f"Error: {e}")

# Test Agent 2 (SystemMessage prompt)
print("\n=== Agent 2 Response (SystemMessage Prompt) ===")
print("Expected: Detailed explanation with research focus")
try:
    result2 = agent2.invoke({"messages": test_question})
    print(f"Response: {result2['messages'][-1].content}")
except Exception as e:
    print(f"Error: {e}")

## Testing Dynamic Prompt with Different User Types

The dynamic prompt agent can adapt based on user context:

In [ ]:
print("=== Agent 3 Responses (Dynamic Prompt) ===")

# Test with expert mode
print("\n--- Expert Mode ---")
print("Expected: Technical response with advanced concepts")
try:
    result3_expert = agent3.invoke({
        "messages": test_question,
        "user_type": "expert"
    })
    print(f"Response: {result3_expert['messages'][-1].content}")
except Exception as e:
    print(f"Error: {e}")

# Test with beginner mode
print("\n--- Beginner Mode ---")
print("Expected: Simple explanation with analogies")
try:
    result3_beginner = agent3.invoke({
        "messages": test_question,
        "user_type": "beginner"
    })
    print(f"Response: {result3_beginner['messages'][-1].content}")
except Exception as e:
    print(f"Error: {e}")

# Test with standard mode (default)
print("\n--- Standard Mode ---")
print("Expected: Balanced explanation")
try:
    result3_standard = agent3.invoke({
        "messages": test_question,
        "user_type": "standard"
    })
    print(f"Response: {result3_standard['messages'][-1].content}")
except Exception as e:
    print(f"Error: {e}")

print("\n💡 Notice how each agent responds differently based on its prompt configuration")

## Advanced Dynamic Prompt Examples

Let's explore more sophisticated dynamic prompt patterns:

In [ ]:
print("=== Advanced Dynamic Prompt Examples ===")

# Context-aware prompt that considers conversation history
def context_aware_prompt(state):
    """Prompt that adapts based on conversation context."""
    messages = state["messages"]
    message_count = len(messages)
    
    # Analyze conversation length
    if message_count > 10:
        style = "You are now in deep conversation mode. Provide comprehensive, detailed responses."
    elif message_count > 5:
        style = "Continue the conversation with informative but concise responses."
    else:
        style = "You are starting a new conversation. Be welcoming and helpful."
    
    # Check for specific topics in recent messages
    recent_content = " ".join(str(msg.content) for msg in messages[-3:] if hasattr(msg, 'content'))
    
    if "technical" in recent_content.lower():
        style += " Focus on technical accuracy and provide examples."
    elif "simple" in recent_content.lower() or "explain" in recent_content.lower():
        style += " Keep explanations simple and clear."
    
    return [SystemMessage(content=style)] + messages

# Time-based prompt
def time_based_prompt(state):
    """Prompt that changes based on time of day (simulated)."""
    import datetime
    hour = datetime.datetime.now().hour
    
    if 6 <= hour < 12:
        greeting = "Good morning! I'm here to help you start your day productively."
    elif 12 <= hour < 18:
        greeting = "Good afternoon! How can I assist you today?"
    else:
        greeting = "Good evening! I'm here to help with any questions you have."
    
    return [SystemMessage(content=greeting)] + state["messages"]

# Domain-specific prompt
def domain_prompt(state):
    """Prompt that adapts based on detected domain."""
    content = " ".join(str(msg.content) for msg in state["messages"] if hasattr(msg, 'content')).lower()
    
    if any(word in content for word in ["code", "programming", "software", "algorithm"]):
        domain_style = "You are a programming assistant. Provide code examples and technical guidance."
    elif any(word in content for word in ["business", "marketing", "strategy", "sales"]):
        domain_style = "You are a business consultant. Focus on practical business advice and strategies."
    elif any(word in content for word in ["science", "research", "study", "experiment"]):
        domain_style = "You are a scientific advisor. Provide evidence-based information and cite sources."
    else:
        domain_style = "You are a general assistant. Provide helpful and accurate information."
    
    return [SystemMessage(content=domain_style)] + state["messages"]

print("✓ Advanced prompt functions defined:")
print("  - context_aware_prompt: Adapts to conversation length and topics")
print("  - time_based_prompt: Changes greeting based on time of day")
print("  - domain_prompt: Detects domain and adjusts expertise")

## Creating Agents with Advanced Prompts

In [ ]:
# Create agents with advanced prompt functions
context_agent = create_agent(model, [tools.helper_tool], prompt=context_aware_prompt)
time_agent = create_agent(model, [tools.helper_tool], prompt=time_based_prompt)
domain_agent = create_agent(model, [tools.helper_tool], prompt=domain_prompt)

print("✓ Advanced agents created")

# Test domain detection
print("\n=== Testing Domain Detection ===")

domain_tests = [
    ("Help me write a Python function", "Programming"),
    ("What's a good marketing strategy?", "Business"),
    ("Explain quantum physics research", "Science"),
    ("What's the weather like?", "General")
]

for query, expected_domain in domain_tests:
    print(f"\nQuery: '{query}'")
    print(f"Expected domain: {expected_domain}")
    try:
        result = domain_agent.invoke({"messages": query})
        print(f"Response type: Detected domain correctly")
    except Exception as e:
        print(f"Error: {e}")

## Prompt Template Patterns

Common patterns for building dynamic prompts:

In [ ]:
print("=== Prompt Template Patterns ===")

# Template-based prompt builder
class PromptBuilder:
    """Builder pattern for constructing dynamic prompts."""
    
    def __init__(self):
        self.base_prompt = "You are a helpful assistant."
        self.personality_traits = []
        self.constraints = []
        self.context_rules = []
    
    def set_personality(self, trait):
        """Add personality trait."""
        self.personality_traits.append(trait)
        return self
    
    def add_constraint(self, constraint):
        """Add behavioral constraint."""
        self.constraints.append(constraint)
        return self
    
    def add_context_rule(self, rule):
        """Add context-specific rule."""
        self.context_rules.append(rule)
        return self
    
    def build(self, state):
        """Build the final prompt based on state."""
        parts = [self.base_prompt]
        
        if self.personality_traits:
            parts.append(f"Personality: {', '.join(self.personality_traits)}.")
        
        if self.constraints:
            parts.append(f"Constraints: {' '.join(self.constraints)}")
        
        if self.context_rules:
            parts.append(f"Context rules: {' '.join(self.context_rules)}")
        
        final_prompt = " ".join(parts)
        return [SystemMessage(content=final_prompt)] + state["messages"]

# Example usage
friendly_prompt = (PromptBuilder()
                   .set_personality("friendly")
                   .set_personality("encouraging")
                   .add_constraint("Keep responses under 100 words.")
                   .add_context_rule("If user seems frustrated, be extra supportive."))

technical_prompt = (PromptBuilder()
                    .set_personality("precise")
                    .set_personality("analytical")
                    .add_constraint("Always provide examples.")
                    .add_constraint("Cite sources when possible.")
                    .add_context_rule("For code questions, include working examples."))

print("✓ Prompt builder pattern implemented")
print("  - Modular prompt construction")
print("  - Reusable prompt components")
print("  - Easy to maintain and extend")

## Testing Prompt Builder Agents

In [ ]:
# Create agents with built prompts
friendly_agent = create_agent(model, [tools.helper_tool], prompt=friendly_prompt.build)
technical_agent = create_agent(model, [tools.helper_tool], prompt=technical_prompt.build)

print("=== Testing Prompt Builder Agents ===")

test_query = "Help me understand machine learning"

print(f"\nTest query: '{test_query}'")

# Test friendly agent
print("\n--- Friendly Agent ---")
print("Expected: Encouraging, supportive response under 100 words")
try:
    friendly_result = friendly_agent.invoke({"messages": test_query})
    response = friendly_result['messages'][-1].content
    print(f"Response ({len(response.split())} words): {response}")
except Exception as e:
    print(f"Error: {e}")

# Test technical agent
print("\n--- Technical Agent ---")
print("Expected: Precise, analytical response with examples")
try:
    technical_result = technical_agent.invoke({"messages": test_query})
    response = technical_result['messages'][-1].content
    print(f"Response: {response}")
except Exception as e:
    print(f"Error: {e}")

## Best Practices Summary

### Method Selection Guide

#### Use String Prompts When:
- **Prototyping** and quick experimentation
- **Simple, static** instructions
- **Learning** LangChain basics
- **Single-purpose** agents

#### Use SystemMessage When:
- **Production** chat applications
- **Structured** message handling
- **Better integration** with chat models
- **Professional** applications

#### Use Dynamic Prompts When:
- **Personalization** is required
- **Context-aware** responses needed
- **Adaptive behavior** based on state
- **Complex** decision logic

### Dynamic Prompt Patterns

1. **User Type Adaptation**
   ```python
   def user_prompt(state):
       user_type = state.get("user_type", "standard")
       # Adjust based on expertise level
   ```

2. **Context History Analysis**
   ```python
   def context_prompt(state):
       recent_topics = analyze_recent_messages(state["messages"])
       # Adapt based on conversation topics
   ```

3. **Domain Detection**
   ```python
   def domain_prompt(state):
       domain = detect_domain(state["messages"])
       # Switch expertise based on detected domain
   ```

4. **Time/Session Context**
   ```python
   def session_prompt(state):
       session_info = state.get("session_info", {})
       # Adapt based on session metadata
   ```

### Design Principles

1. **Progressive Enhancement**: Start simple, add complexity as needed
2. **Separation of Concerns**: Keep prompt logic separate from business logic
3. **Testability**: Make dynamic prompts easy to test with different inputs
4. **Maintainability**: Use builder patterns for complex prompt construction
5. **Performance**: Cache expensive prompt computations when possible

### Common Pitfalls

1. **Over-complexity**: Don't make prompts unnecessarily complex
2. **Inconsistent Behavior**: Ensure dynamic prompts produce consistent results
3. **Poor Error Handling**: Handle edge cases in dynamic prompt functions
4. **Lack of Fallbacks**: Always have default behavior for unexpected states
5. **Prompt Injection**: Validate and sanitize dynamic content

## Conclusion

Prompt configuration is crucial for agent behavior:
- **String prompts** are perfect for getting started
- **SystemMessage** provides structure for production use
- **Dynamic prompts** enable sophisticated, adaptive agents
- **Builder patterns** help manage complex prompt logic
- **Context awareness** dramatically improves user experience

Choose the method that best fits your use case, and don't be afraid to start simple and evolve your approach as your requirements grow.